In [3]:
#Acá agrupamos los diferentes archivos .json y los limpiamos de filas vacias y espacios para guardralo en un archivo .csv de 259.700 casos
#No hace falta correr este código xq ya tenemos el .csv con la info guardada en "tweets_filtrados_combinados.csv"

import os
import json
import pandas as pd

# Definir las rutas de las carpetas
folder_paths = [
    r"C:\Users\usuario\Desktop\TwitterMigration\TW_DNU_original\TW_DNU_1",
    r"C:\Users\usuario\Desktop\TwitterMigration\TW_DNU_original\TW_DNU_2"
]

# Lista para almacenar los datos combinados
all_data = []

# Procesar archivos en cada carpeta
for folder_path in folder_paths:
    print(f"Procesando carpeta: {folder_path}")
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".json"):  # Filtrar solo archivos JSON
            file_path = os.path.join(folder_path, file_name)  # Ruta completa al archivo
            print(f"Procesando archivo: {file_name}")
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    for line in f:
                        line = line.strip()  # Eliminar espacios en blanco al inicio y al final
                        if line:  # Solo procesar líneas no vacías
                            try:
                                obj = json.loads(line)  # Intentar cargar como JSON
                                all_data.append(obj)  # Agregar el objeto JSON a la lista
                            except json.JSONDecodeError as e:
                                print(f"Error al decodificar una línea en {file_name}: {e}")
            except FileNotFoundError:
                print(f"El archivo {file_name} no se encontró en la ruta especificada.")

# Convertir los datos combinados en un DataFrame
if all_data:
    df = pd.json_normalize(all_data)
    print(f"\nDatos combinados: {len(df)} filas procesadas.")
else:
    print("No se encontraron datos válidos en los archivos.")

# Filtrar columnas relevantes
if not df.empty:
    relevant_columns = [
        'created_at', 
        'id', 
        'id_str', 
        'text', 
        'source', 
        'truncated', 
        'in_reply_to_status_id', 
        'in_reply_to_status_id_str', 
        'in_reply_to_user_id', 
        'quoted_status.place.place_type', 
        'quoted_status.place.name', 
        'retweeted_status.quoted_status.quoted_status_id'
    ]
    # Verificar si las columnas están en el DataFrame
    filtered_columns = [col for col in relevant_columns if col in df.columns]
    filtered_df = df[filtered_columns]

    # Mostrar las primeras filas del DataFrame filtrado
    print("\nPrimeras filas del DataFrame filtrado:")
    print(filtered_df.head())
else:
    print("El DataFrame está vacío.")

# Guardar el DataFrame filtrado en un archivo CSV
output_file = "tweets_filtrados_combinados.csv"
filtered_df.to_csv(output_file, index=False, encoding='utf-8')
print(f"\nArchivo guardado como: {output_file}")



Procesando carpeta: C:\Users\usuario\Desktop\TwitterMigration\TW_DNU_original\TW_DNU_1
Procesando archivo: DNU 05-03-2021 08 01 .json
Procesando archivo: DNU 05-03-2021 08 06 .json
Procesando archivo: DNU 05-03-2021 08 13 .json
Error al decodificar una línea en DNU 05-03-2021 08 13 .json: Unterminated string starting at: line 1 column 3313 (char 3312)
Error al decodificar una línea en DNU 05-03-2021 08 13 .json: Expecting value: line 1 column 1 (char 0)
Error al decodificar una línea en DNU 05-03-2021 08 13 .json: Unterminated string starting at: line 1 column 3540 (char 3539)
Error al decodificar una línea en DNU 05-03-2021 08 13 .json: Expecting value: line 1 column 1 (char 0)
Error al decodificar una línea en DNU 05-03-2021 08 13 .json: Unterminated string starting at: line 1 column 9752 (char 9751)
Error al decodificar una línea en DNU 05-03-2021 08 13 .json: Expecting value: line 1 column 1 (char 0)
Error al decodificar una línea en DNU 05-03-2021 08 13 .json: Unterminated string 

In [1]:
#Análisis exploratorio de los datos
import os
import pandas as pd
#Cargar el archivo CSV
file_path = "tweets_filtrados_combinados.csv"  #
df = pd.read_csv(file_path)

# Mostrar información básica
print("Resumen del DataFrame:")
print(df.info())

print("\nPrimeras filas del DataFrame:")
print(df.head())

Resumen del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259752 entries, 0 to 259751
Data columns (total 12 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   created_at                                       259752 non-null  object 
 1   id                                               259752 non-null  int64  
 2   id_str                                           259752 non-null  int64  
 3   text                                             259752 non-null  object 
 4   source                                           259734 non-null  object 
 5   truncated                                        259752 non-null  bool   
 6   in_reply_to_status_id                            34274 non-null   float64
 7   in_reply_to_status_id_str                        34274 non-null   float64
 8   in_reply_to_user_id                              35700 non-null   float

In [2]:
# Detectar retweets basados en el texto que comienza con "RT"
if 'text' in df.columns:
    df['is_retweet'] = df['text'].str.startswith('RT')  # Crear una columna booleana
    total_retweets = df['is_retweet'].sum()  # Contar retweets
    print(f"Total de retweets: {total_retweets}")
    print(f"Porcentaje de retweets: {total_retweets / len(df) * 100:.2f}%")
else:
    print("La columna 'text' no está en el DataFrame.")

Total de retweets: 196293
Porcentaje de retweets: 75.57%


In [3]:
#Distribución de Retweets y Tweets Originales
retweets_v_original = df['is_retweet'].value_counts()
print("\nDistribución de Retweets vs Tweets Originales:")
print(retweets_v_original)


Distribución de Retweets vs Tweets Originales:
is_retweet
True     196293
False     63459
Name: count, dtype: int64


In [9]:
# Filtrar solo retweets
retweets = df[df['is_retweet']]

# Filtrar solo tweets originales
tweets_originales = df[~df['is_retweet']]

print(f"Total de retweets filtrados: {len(retweets)}")
print(f"Total de tweets originales filtrados: {len(tweets_originales)}")

Total de retweets filtrados: 196293
Total de tweets originales filtrados: 63459


In [4]:
#contar cuántos caracteres posee cada
def contar_caracteres(texto):
    return len(texto)

df["caracteres"] = df["text"].apply(contar_caracteres)

df

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,quoted_status.place.place_type,quoted_status.place.name,retweeted_status.quoted_status.quoted_status_id,is_retweet,caracteres
0,Fri Mar 05 11:01:46 +0000 2021,1367792447334580225,1367792447334580225,RT @CELS_Argentina: ✅Celebramos la decisión de...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140
1,Fri Mar 05 11:06:31 +0000 2021,1367793643239702529,1367793643239702529,RT @LANACION: Ley de Migraciones: el Gobierno ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,139
2,Fri Mar 05 11:06:33 +0000 2021,1367793651192070144,1367793651192070144,"RT @CRLDEMONIO: Y lo peor , el alto mando d...","<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,139
3,Fri Mar 05 11:06:34 +0000 2021,1367793655092813824,1367793655092813824,RT @elcarpo: El Gobierno derogó el decreto de ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140
4,Fri Mar 05 11:06:34 +0000 2021,1367793655029764101,1367793655029764101,@joseantoniokast @sebastianpinera yo voté por ...,"<a href=""http://twitter.com/download/android"" ...",True,1.367503e+18,1.367503e+18,123955962.0,NaN,NaN,NaN,False,140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259747,Tue Mar 09 00:02:18 +0000 2021,1369076039259987968,1369076039259987968,RT @AsktoMandrake: Te confieso que admiro como...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140
259748,Tue Mar 09 00:02:19 +0000 2021,1369076041482960899,1369076041482960899,RT @MoroBenitez: Sería interesante hacer un hi...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140
259749,Tue Mar 09 00:02:20 +0000 2021,1369076045324972032,1369076045324972032,@Andynaxo @Lady_Chiyome @Ejercito_Chile @katym...,"<a href=""http://twitter.com/download/android"" ...",True,1.369074e+18,1.369074e+18,349289160.0,NaN,NaN,NaN,False,140
259750,Tue Mar 09 00:02:20 +0000 2021,1369076045954113536,1369076045954113536,@canalN_ Es lo mejor porque ya existen hasta d...,"<a href=""http://twitter.com/download/android"" ...",False,1.369067e+18,1.369067e+18,248692867.0,NaN,NaN,NaN,False,103


In [13]:
#promedio de caracteres
df["caracteres"].describe()

count    259752.000000
mean        127.943250
std          25.460652
min           4.000000
25%         132.000000
50%         140.000000
75%         140.000000
max         152.000000
Name: caracteres, dtype: float64

In [14]:
#Podemos ver que la cantidad mínima de caracteres, la máxima y los percentiles 25, 50 y 75.
#Vemos que el mínimo se alejan significativamente de la media de caracteres.
#Estos datos se consideran outliers. Para ver cuáles son los tweets más cortos y más largos, podemos ordenar el DataFrame por la columna caracteres usando el método sort_values() de pandas:

df.sort_values(by="caracteres")

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,quoted_status.place.place_type,quoted_status.place.name,retweeted_status.quoted_status.quoted_status_id,is_retweet,caracteres
176904,Sat Mar 06 21:14:40 +0000 2021,1368309074975068162,1368309074975068162,Amén,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,NaN,NaN,False,4
101790,Fri Mar 05 23:22:16 +0000 2021,1367978801783783424,1367978801783783424,TUGO,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,NaN,NaN,False,4
111232,Sat Mar 06 00:33:31 +0000 2021,1367996732135587842,1367996732135587842,pero,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,False,4
3502,Fri Mar 05 12:20:47 +0000 2021,1367812333993594889,1367812333993594889,jaja,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,NaN,NaN,False,4
60292,Fri Mar 05 18:27:40 +0000 2021,1367904660024098824,1367904660024098824,TUGO,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,NaN,NaN,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81621,Fri Mar 05 20:51:04 +0000 2021,1367940750760157188,1367940750760157188,RT @ElsaKekita: &amp;#8220;Favorece el crimen ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,148
191789,Sun Mar 07 01:05:47 +0000 2021,1368367238630277125,1368367238630277125,RT @soychavezradica: @metro_caracas &lt;&lt;&l...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,152
192152,Sun Mar 07 01:10:24 +0000 2021,1368368402344140806,1368368402344140806,RT @soychavezradica: @metro_caracas &lt;&lt;&l...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,152
194062,Sun Mar 07 01:35:44 +0000 2021,1368374775295381506,1368374775295381506,RT @soychavezradica: @metro_caracas &lt;&lt;&l...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,152


In [15]:
print(df.iloc[191717]["text"])

@metro_caracas &lt;&lt;&lt;&lt;JAJAJAJA ESTOS SINVERGUENZAS Y DELINCUENTES ME TIENEN BLOQUEADO X MIS CONSTANTES DENUNCIAS EN CU… https://t.co/f7Tzwd3luH


In [8]:
#Preprocesamiento 
#!pip install spacy
#!python -m spacy download es_core_news_sm
#!pip install spanish-nlp
#!pip install unidecode

In [7]:
import spacy
nlp = spacy.load("es_core_news_sm")

In [9]:
from spanish_nlp import SpanishPreprocess

sp = SpanishPreprocess(
    lower=True,
    remove_url=True,
    remove_hashtags=False,
    split_hashtags=True,
    normalize_breaklines=True,
    remove_emoticons=False,
    remove_emojis=False,
    convert_emoticons=False,
    convert_emojis=False,
    normalize_inclusive_language=True,
    reduce_spam=True,
    remove_vowels_accents=True,
    remove_multiple_spaces=True,
    remove_punctuation=True,
    remove_unprintable=True,
    remove_numbers=True,
    remove_stopwords=False,
    stopwords_list=None,
    lemmatize=False,
    stem=False,
    remove_html_tags=True,
)

In [10]:
spacy.lang.es.stop_words.STOP_WORDS
df["text_pp"] = df["text"].apply(lambda x: sp.transform(x)) #eliminar caracteres especiales, URLs, números y para normalizar palabra

In [12]:
def tokenizar(text):
    doc = nlp(text)
    return [
        token.lemma_ for token in doc if not token.is_punct and not token.is_stop and token.is_alpha
    ]


df["tokens"] = df["text_pp"].apply(tokenizar)
df

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,quoted_status.place.place_type,quoted_status.place.name,retweeted_status.quoted_status.quoted_status_id,is_retweet,caracteres,text_pp,tokens
0,Fri Mar 05 11:01:46 +0000 2021,1367792447334580225,1367792447334580225,RT @CELS_Argentina: ✅Celebramos la decisión de...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140,rt cels_argentina ✅ celebramos la decision del...,"[rt, celebramos, decision, ejecutivo, derogar,..."
1,Fri Mar 05 11:06:31 +0000 2021,1367793643239702529,1367793643239702529,RT @LANACION: Ley de Migraciones: el Gobierno ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,139,rt lanacion ley de migraciones el gobierno der...,"[rt, lanacion, ley, migración, gobierno, derog..."
2,Fri Mar 05 11:06:33 +0000 2021,1367793651192070144,1367793651192070144,"RT @CRLDEMONIO: Y lo peor , el alto mando d...","<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,139,rt crldemonio y lo peor el alto mando de carab...,"[rt, crldemonio, alto, mando, carabdechile, ap..."
3,Fri Mar 05 11:06:34 +0000 2021,1367793655092813824,1367793655092813824,RT @elcarpo: El Gobierno derogó el decreto de ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140,rt elcarpo el gobierno derogo el decreto de ma...,"[rt, elcarpo, gobierno, derogo, decreto, mauri..."
4,Fri Mar 05 11:06:34 +0000 2021,1367793655029764101,1367793655029764101,@joseantoniokast @sebastianpinera yo voté por ...,"<a href=""http://twitter.com/download/android"" ...",True,1.367503e+18,1.367503e+18,123955962.0,NaN,NaN,NaN,False,140,joseantoniokast sebastianpinera yo vote por se...,"[joseantoniokast, sebastianpinero, votar, seba..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259747,Tue Mar 09 00:02:18 +0000 2021,1369076039259987968,1369076039259987968,RT @AsktoMandrake: Te confieso que admiro como...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140,rt asktomandrake te confieso que admiro como l...,"[rt, asktomandrake, confesar, admirar, extorsi..."
259748,Tue Mar 09 00:02:19 +0000 2021,1369076041482960899,1369076041482960899,RT @MoroBenitez: Sería interesante hacer un hi...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140,rt morobenitez seria interesante hacer un hilo...,"[rt, morobenitez, serio, interesante, hilo, pe..."
259749,Tue Mar 09 00:02:20 +0000 2021,1369076045324972032,1369076045324972032,@Andynaxo @Lady_Chiyome @Ejercito_Chile @katym...,"<a href=""http://twitter.com/download/android"" ...",True,1.369074e+18,1.369074e+18,349289160.0,NaN,NaN,NaN,False,140,andynaxo lady_chiyome ejercito_chile katymarto...,"[andynaxo, katymartorell, estatua, weon, uds, ..."
259750,Tue Mar 09 00:02:20 +0000 2021,1369076045954113536,1369076045954113536,@canalN_ Es lo mejor porque ya existen hasta d...,"<a href=""http://twitter.com/download/android"" ...",False,1.369067e+18,1.369067e+18,248692867.0,NaN,NaN,NaN,False,103,canaln_ es lo mejor porque ya existen hasta de...,"[canaln, delincuente, hurtir, informacion, adu..."


In [14]:
# Guardar el DataFrame filtrado en un archivo CSV
output_file = "tweets_filtrados_combinados.csv"
df.to_csv(output_file, index=False, encoding='utf-8')
print(f"\nArchivo guardado como: {output_file}")


Archivo guardado como: tweets_filtrados_combinados.csv


In [15]:
print(df.head())  # Para ver las primeras filas del DataFrame


                       created_at                   id               id_str  \
0  Fri Mar 05 11:01:46 +0000 2021  1367792447334580225  1367792447334580225   
1  Fri Mar 05 11:06:31 +0000 2021  1367793643239702529  1367793643239702529   
2  Fri Mar 05 11:06:33 +0000 2021  1367793651192070144  1367793651192070144   
3  Fri Mar 05 11:06:34 +0000 2021  1367793655092813824  1367793655092813824   
4  Fri Mar 05 11:06:34 +0000 2021  1367793655029764101  1367793655029764101   

                                                text  \
0  RT @CELS_Argentina: ✅Celebramos la decisión de...   
1  RT @LANACION: Ley de Migraciones: el Gobierno ...   
2  RT @CRLDEMONIO: Y lo peor  ,   el alto mando d...   
3  RT @elcarpo: El Gobierno derogó el decreto de ...   
4  @joseantoniokast @sebastianpinera yo voté por ...   

                                              source  truncated  \
0  <a href="http://twitter.com/download/android" ...      False   
1  <a href="http://twitter.com/download/android" ...  

In [20]:
import os
import pandas as pd
#Cargar el archivo CSV
file_path = "tweets_filtrados_combinados.csv"  
df = pd.read_csv(file_path)

df

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,quoted_status.place.place_type,quoted_status.place.name,retweeted_status.quoted_status.quoted_status_id,is_retweet,caracteres,text_pp,tokens
0,Fri Mar 05 11:01:46 +0000 2021,1367792447334580225,1367792447334580225,RT @CELS_Argentina: ✅Celebramos la decisión de...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140,rt cels_argentina ✅ celebramos la decision del...,"['rt', 'celebramos', 'decision', 'ejecutivo', ..."
1,Fri Mar 05 11:06:31 +0000 2021,1367793643239702529,1367793643239702529,RT @LANACION: Ley de Migraciones: el Gobierno ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,139,rt lanacion ley de migraciones el gobierno der...,"['rt', 'lanacion', 'ley', 'migración', 'gobier..."
2,Fri Mar 05 11:06:33 +0000 2021,1367793651192070144,1367793651192070144,"RT @CRLDEMONIO: Y lo peor , el alto mando d...","<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,139,rt crldemonio y lo peor el alto mando de carab...,"['rt', 'crldemonio', 'alto', 'mando', 'carabde..."
3,Fri Mar 05 11:06:34 +0000 2021,1367793655092813824,1367793655092813824,RT @elcarpo: El Gobierno derogó el decreto de ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140,rt elcarpo el gobierno derogo el decreto de ma...,"['rt', 'elcarpo', 'gobierno', 'derogo', 'decre..."
4,Fri Mar 05 11:06:34 +0000 2021,1367793655029764101,1367793655029764101,@joseantoniokast @sebastianpinera yo voté por ...,"<a href=""http://twitter.com/download/android"" ...",True,1.367503e+18,1.367503e+18,123955962.0,NaN,NaN,NaN,False,140,joseantoniokast sebastianpinera yo vote por se...,"['joseantoniokast', 'sebastianpinero', 'votar'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259747,Tue Mar 09 00:02:18 +0000 2021,1369076039259987968,1369076039259987968,RT @AsktoMandrake: Te confieso que admiro como...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140,rt asktomandrake te confieso que admiro como l...,"['rt', 'asktomandrake', 'confesar', 'admirar',..."
259748,Tue Mar 09 00:02:19 +0000 2021,1369076041482960899,1369076041482960899,RT @MoroBenitez: Sería interesante hacer un hi...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,NaN,NaN,NaN,NaN,NaN,True,140,rt morobenitez seria interesante hacer un hilo...,"['rt', 'morobenitez', 'serio', 'interesante', ..."
259749,Tue Mar 09 00:02:20 +0000 2021,1369076045324972032,1369076045324972032,@Andynaxo @Lady_Chiyome @Ejercito_Chile @katym...,"<a href=""http://twitter.com/download/android"" ...",True,1.369074e+18,1.369074e+18,349289160.0,NaN,NaN,NaN,False,140,andynaxo lady_chiyome ejercito_chile katymarto...,"['andynaxo', 'katymartorell', 'estatua', 'weon..."
259750,Tue Mar 09 00:02:20 +0000 2021,1369076045954113536,1369076045954113536,@canalN_ Es lo mejor porque ya existen hasta d...,"<a href=""http://twitter.com/download/android"" ...",False,1.369067e+18,1.369067e+18,248692867.0,NaN,NaN,NaN,False,103,canaln_ es lo mejor porque ya existen hasta de...,"['canaln', 'delincuente', 'hurtir', 'informaci..."


In [28]:
import ast

# Convertir las cadenas en listas reales si es necesario
if isinstance(df["tokens"].iloc[0], str):  # Verifica si el primer elemento es una cadena
    df["tokens"] = df["tokens"].apply(ast.literal_eval)
    

from collections import Counter

# Filtrar tweets originales
df_tweets = df[df["is_retweet"] == False]

# Verificar que no haya valores nulos en 'tokens'
df_tweets = df_tweets.dropna(subset=["tokens"])

# Unir todos los tokens en una lista desde los casos filtrados
all_tokens = [token for tokens in df_tweets["tokens"] for token in tokens]

# Contar la frecuencia de las palabras
word_freq = Counter(all_tokens)

# Obtener las 30 palabras más frecuentes
top_words = word_freq.most_common(30)

# Mostrar las 30 palabras más frecuentes
print("30 palabras más frecuentes en tweets originales:")
for word, freq in top_words:
    print(f"{word}: {freq}")


30 palabras más frecuentes en tweets originales:
delincuente: 25027
extranjero: 6843
gobierno: 4097
antecedente: 3584
q: 3570
pais: 2868
alferdez: 2779
dejar: 1992
entrar: 1902
expulsion: 1890
decreto: 1864
ingreso: 1586
gente: 1520
mujer: 1500
querer: 1381
d: 1345
defender: 1308
pai: 1244
eligalcorto: 1162
macri: 1143
persona: 1102
narco: 1089
ver: 1000
seguir: 993
ley: 975
corrupto: 967
permitir: 956
politico: 956
año: 926
pasar: 925


In [31]:
#Los n-gramas son útiles para detectar frases o expresiones que se repiten en el corpus. Para ello, vamos a utilizar CountVectorizer de scikit-learn:
from sklearn.feature_extraction.text import CountVectorizer

stopwords = list(spacy.lang.es.stop_words.STOP_WORDS)

# Crear instancia de CountVectorizer
vectorizer = CountVectorizer(ngram_range=(2, 2), stop_words=stopwords)

## Crear matriz de conteo de n-gramas
ngram_matrix = vectorizer.fit_transform(df_tweets["text_pp"])

ngram_matrix

<63459x286201 sparse matrix of type '<class 'numpy.int64'>'
	with 401550 stored elements in Compressed Sparse Row format>

In [32]:
#Finalmente, vamos a sumar las frecuencias de los n-gramas y a mostrar los n-gramas más comunes.
import numpy as np

# Sumar las frecuencias de los n-gramas
ngram_freq = np.sum(ngram_matrix, axis=0)

# Obtener los n-gramas más comunes
top_ngrams = [(ngram, ngram_freq[0, index]) for ngram, index in vectorizer.vocabulary_.items()]
top_ngrams = sorted(top_ngrams, key=lambda x: x[1], reverse=True)[:30]

print(top_ngrams)

[('eligalcorta alferdez', 1141), ('extranjeros antecedentes', 1045), ('antecedentes penalos', 788), ('delincuentes extranjeros', 607), ('derogo decreto', 560), ('gobierno delincuentes', 555), ('impedia ingreso', 524), ('decreto macri', 488), ('gobierno derogo', 410), ('ingreso extranjeros', 385), ('pais extranjeros', 367), ('ingreso pais', 366), ('derechos humanos', 364), ('entrar delincuentes', 334), ('macri impedia', 319), ('delincuentes pais', 291), ('dejar entrar', 286), ('banda delincuentes', 243), ('delincuentes terroristas', 240), ('delincuentes gobierno', 226), ('pais delincuentes', 225), ('decreto impedia', 216), ('delincuentes asesinos', 213), ('alferdez alferdezprensa', 211), ('delincuentes narcos', 208), ('dejan entrar', 204), ('narcos delincuentes', 192), ('alferdezprensa promediando', 192), ('promediando abajo', 191), ('permitiendo ingreso', 190)]


In [33]:
#Vectorizacion
#Para poder hacer análisis más profundos de los textos, necesitamos representarlos en forma numérica. Este proceso se conoce como vectorización, es decir, transformar los textos en vectores (secuencias de números).
#Existen diversas formas de vectorizar textos. En esta clase vamos a ver dos de ellas: bag-of-words y TF-IDF.
#También, podemos contar la frecuencia de los n-gramas en cada texto. Para ello, nuevamente, vamos a utilizar CountVectorizer de scikit-learn. Esta vez con el parámetro ngram_range (1, 1) (por defecto) para contar palabras y con el parámetro ngram_range (1, 2) para contar palabras y 2-gramas.
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_by_word = CountVectorizer(ngram_range=(1, 1), stop_words=stopwords)

# Entrenamos el vectorizador
vectorizer_by_word.fit(df_tweets["text_pp"])


CountVectorizer(stop_words=['fuimos', 'ello', 'mencionó', 'esas', 'conmigo',
                            'muchos', 'dicen', 'ha', 'los', 'cuenta', 'tres',
                            'todavia', 'tercero', 'vaya', 'lo', 'todos', 'cómo',
                            'fin', 'luego', 'otra', 'pero', 'casi', 'once',
                            'total', 'mios', 'señaló', 'días', 'sobre',
                            'cuanta', 'habían', ...])

In [34]:
vectorizer_by_2gram = CountVectorizer(ngram_range=(1, 2), max_features=1000, stop_words=stopwords)

# Entrenamos el vectorizador
vectorizer_by_2gram.fit(df_tweets["text_pp"])

CountVectorizer(max_features=1000, ngram_range=(1, 2),
                stop_words=['fuimos', 'ello', 'mencionó', 'esas', 'conmigo',
                            'muchos', 'dicen', 'ha', 'los', 'cuenta', 'tres',
                            'todavia', 'tercero', 'vaya', 'lo', 'todos', 'cómo',
                            'fin', 'luego', 'otra', 'pero', 'casi', 'once',
                            'total', 'mios', 'señaló', 'días', 'sobre',
                            'cuanta', 'habían', ...])

In [35]:
corpus_vectorizado_bow = vectorizer_by_2gram.transform(df["text_pp"])

# Lo convertimos a un DataFrame con los nombres de las columnas
df_corpus_vectorizado_bow = pd.DataFrame(
    corpus_vectorizado_bow.toarray(), columns=vectorizer_by_2gram.get_feature_names_out()
)
df_corpus_vectorizado_bow

,__sparklos__,abajo,abajo permitiendo,abogado,abrahamendieta,abre,abren,abrir,aca,acaso,...,votaron,voten,voto,votos,vuelve,weones,willaxtv,wolffwaldo,xq,zurdos
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259747,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
259748,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
259749,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
259750,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
#Aunque puede parecer una vectorización sencilla y poco sofisticada, bag-of-words es una técnica bastante útil ya que logra capturar el significado de los textos. Por ejemplo, si tenemos dos textos que hablan de fútbol, es probable que compartan palabras como gol, equipo, jugador, partido, etc. Por lo tanto, es probable que los vectores de estos textos sean similares. Por ejemplo, podemos buscar el texto que más se parece a texto_prueba.
#Para ello recorremos todos los textos del corpus y calculamos la similitud coseno con el texto de prueba. La similitud coseno es una medida de similitud entre dos vectores que mide el coseno del ángulo entre ellos. Si los vectores son idénticos, la similitud coseno es 1. Si los vectores son ortogonales, la similitud coseno es 0. Si los vectores son opuestos, la similitud coseno es -1.
#Para calcular la similitud coseno, vamos a utilizar la función cosine_similarity de scikit-learn:
from sklearn.metrics.pairwise import cosine_similarity

# Hay que calcular la similitud entre el texto de prueba y todos los demás.
matriz_similitud = cosine_similarity(corpus_vectorizado_bow[100], corpus_vectorizado_bow)

# Ordenar los textos por similitud
textos_similares = list(zip(df_tweets["text_pp"], matriz_similitud[0]))
textos_similares.sort(key=lambda x: -x[1])

# Mostrar los 5 textos más similares
for texto, similitud in textos_similares[:6]:
    print(f"Similitud: {similitud}\n{texto}\n")

Similitud: 0.0
joseantoniokast sebastianpinera yo vote por sebastianpinera 
y estoy entre los millones de chilenos que se sien

Similitud: 0.0
todonoticias derogo el dnu que impedia la entrada al pais de delincuentes periodistas al servicio de la deformacion informativa

Similitud: 0.0
derogan el decreto de macri que impedia el ingreso al pais de extranjeros con antecedentes

Similitud: 0.0
paochicapaopao elcerditotv lexisalmansa alexcolo_ giulitruran nacionalidad y las verdes son extranjeros i

Similitud: 0.0
todonoticias que verguenza da le queda gigante el cargo habla de seguridad y su gobierno acaba de derogar la

Similitud: 0.0
vengan los delincuentes del mundo vengan nomas aca los recibimos con los brazos abiertos y los votos asegurados



In [37]:
#Extraer tópicos de los textos
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10, random_state=0)

lda

LatentDirichletAllocation(random_state=0)

In [38]:
# Entrenamos LDA
lda.fit(corpus_vectorizado_tfidf)

LatentDirichletAllocation(random_state=0)

In [39]:
# Podemos obtener los tópicos con el método `components_` de la instancia de LDA:

topicos = lda.components_
topicos

array([[1.00001796e-01, 1.00000000e-01, 1.00000419e-01, ...,
        1.00000000e-01, 1.00000000e-01, 1.00000000e-01],
       [1.00001925e-01, 1.00000000e-01, 1.00011767e-01, ...,
        1.00000000e-01, 1.00000087e-01, 1.00000000e-01],
       [1.00000319e-01, 1.00000000e-01, 1.00001261e-01, ...,
        1.00000000e-01, 1.00000245e-01, 1.00000000e-01],
       ...,
       [1.00000785e-01, 1.00000000e-01, 1.00001098e-01, ...,
        1.00000000e-01, 1.00000679e-01, 1.00000000e-01],
       [3.02943081e+02, 1.87196168e+02, 3.01676425e+02, ...,
        1.00000000e-01, 1.75523492e+02, 1.64970599e+02],
       [1.00000626e-01, 1.00000000e-01, 1.00000388e-01, ...,
        1.00000000e-01, 1.00000000e-01, 1.00000000e-01]])

In [40]:
#Tenemos una matriz de 10 tópicos y 1000 palabras.
#Vamos a mostrar las 20 palabras que componen cada tópico:
# Obtener las palabras de cada tópico
words = vectorizer_by_2gram.get_feature_names_out()

top_words_per_topic = []

for numero_topico in range(10):
    print(f"\nTópico {numero_topico + 1}")
    print("-" * 50)

    # Obtener los pesos de las palabras para este tópico
    pesos_palabras = topicos[numero_topico]

    # Crear lista de tuplas (palabra, peso)
    palabras_y_pesos = list(zip(words, pesos_palabras))

    # Ordenar por peso de mayor a menor
    palabras_y_pesos = sorted(palabras_y_pesos, key=lambda x: x[1], reverse=True)

    # Mostrar las 20 primeras palabras
    for palabra, peso in palabras_y_pesos[:10]:
        print(f"Palabra: {palabra:<20} Peso: {peso:.4f}")


Tópico 1
--------------------------------------------------
Palabra: recibir              Peso: 1348.5188
Palabra: boca                 Peso: 1339.8290
Palabra: delincuentes mundo   Peso: 1156.4369
Palabra: delincuente          Peso: 1129.9019
Palabra: socialos             Peso: 1018.6177
Palabra: estatua              Peso: 967.1081
Palabra: esten                Peso: 949.9027
Palabra: fueran               Peso: 908.7772
Palabra: delincuentes         Peso: 897.5991
Palabra: kks lopezobrador_    Peso: 855.4181

Tópico 2
--------------------------------------------------
Palabra: recibir              Peso: 1554.8699
Palabra: eligalcorta          Peso: 1137.5149
Palabra: obvio                Peso: 1108.0404
Palabra: alberto              Peso: 1059.3158
Palabra: delincuentes mundo   Peso: 983.9838
Palabra: ganar                Peso: 936.5267
Palabra: pagina               Peso: 904.6667
Palabra: cualos               Peso: 874.2689
Palabra: gobierno delincuentes Peso: 853.8649
Palabra: esta

In [41]:
#Extracción de Tópicos con BERTopic
#LDA (Latent Dirichlet Allocation) es un método popular para la extracción de tópicos, pero presenta algunas limitaciones:

#LDA supone que los documentos se generan a partir de una mezcla de tópicos y que cada tópico es una mezcla de palabras, lo cual puede no ser cierto en todos los casos.
#LDA no siempre maneja bien documentos cortos o con tópicos poco definidos.
#BERTopic: un enfoque más moderno
#BERTopic utiliza modelos de lenguaje basados en Transformers (como BERT) para generar representaciones vectoriales de textos (embeddings). Luego, agrupa estos vectores para identificar tópicos, lo que ofrece ventajas sobre LDA:

#No asume una distribución específica de palabras en los tópicos.
#Capta mejor el contexto semántico de los textos.
#Es más robusto con textos cortos
#Instalación de BERTopic
#Para comenzar, instalamos la librería con el siguiente comando:

#!pip install bertopic

   ---------------------------------------- 0.0/143.7 kB ? eta -:--:--
   ---------------------------------------- 143.7/143.7 kB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/732.2 kB ? eta -:--:--
   --------------------------- ----------- 522.2/732.2 kB 10.9 MB/s eta 0:00:01
   ---------------------------------------- 732.2/732.2 kB 9.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/275.7 kB ? eta -:--:--
   ------------------------------------- - 266.2/275.7 kB 16.0 MB/s eta 0:00:01
   ---------------------------------------- 275.7/275.7 kB 8.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/88.8 kB ? eta -:--:--
   ---------------------------------------- 88.8/88.8 kB 4.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/56.9 kB ? eta -:--:--
   ---------------------------------------- 56.9/56.9 kB ? eta 0:00:00


In [42]:

##########PAGAMOS O NO???

from bertopic import BERTopic

# Crear una instancia del modelo BERTopic
topic_model = BERTopic(
    language="multilingual", embedding_model="paraphrase-multilingual-MiniLM-L12-v2", verbose=True
)

AttributeError: module 'openai' has no attribute 'OpenAI'

In [ ]:
# Entrenar el modelo y obtener los tópicos
topics, probs = topic_model.fit_transform(df["text"])

# Obtener información detallada de los tópicos
topic_info = topic_model.get_topic_info()
topic_info

In [ ]:
# Asignar tópicos a los textos
df["topico_bertopic"] = topics
df

In [43]:
#Modelos preentrenados con HuggingFace
#HuggingFace es una librería que permite acceder a modelos preentrenados de lenguaje natural. En particular, vamos a utilizar modelos preentrenados para clasificar textos en las siguientes categorías:
#Análisis de sentimientos y Hate speech
#Primero vamos a instalar la librería transformers de HuggingFace:
!pip install transformers

In [44]:
from transformers import pipeline

# Crea una instancia del pipeline de análisis de sentimiento
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="pysentimiento/robertuito-sentiment-analysis",
    return_all_scores=True,
)

config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [45]:
# Crea una instancia del pipeline de análisis de hate
hate_analyzer = pipeline(
    "text-classification",
    model="pysentimiento/robertuito-hate-speech",
    return_all_scores=True,
)


config.json:   0%|          | 0.00/956 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [46]:
df["sentimiento"] = df["text"].apply(lambda x: sentiment_analyzer(x))
df["hate_speech"] = df["text"].apply(lambda x: hate_analyzer(x))


KeyboardInterrupt



In [48]:
# Guardar el DataFrame filtrado en un archivo CSV
output_file = "tweets_filtrados_combinados.csv"
df.to_csv(output_file, index=False, encoding='utf-8')